In [52]:
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple
import csv


def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-2]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_bandwidth(network_snapshots: List[Path]) -> Dict[int, Tuple[int, int]]:
	"""extract relay average in/out bandwidths for PID"""
	usage = dict()
	for snapshot in network_snapshots:
		with snapshot.open() as ssfile:
			reader = csv.reader(ssfile)
			for reading in reader:
				if reading[1].startswith("moq-relay."):
					pid = int(reading[1].replace("moq-relay.", ""))
					time = datetime.fromisoformat(datetime.today().isoformat()[:11]+reading[0])
					# full date doesn't really matter anyway
					# unless the measurement unless the measurement was done overnight
					bytes_in = int(reading[2])
					bytes_out = int(reading[3])
					if pid not in usage:
						usage[pid] = [(time, bytes_in, bytes_out)]
					else:
						usage[pid].append((time, bytes_in, bytes_out))
	# average the readings over time
	# expecting just two readings here, must be updated for more readings
	avg = dict()
	for pid, data in usage.items():
		t = abs((data[0][0] - data[1][0]).total_seconds())
		d_in = abs(data[0][1] - data[1][1])
		d_out = abs(data[0][2] - data[1][2])
		avg[pid] = (d_in / t , d_out / t)

	return avg

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((datetime.fromisoformat(ts.strip()[:-1]), float(val.strip())))

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	percentages = [v for _, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []
	network_snapshots = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		elif file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)
		elif file.name.startswith("network-snapshot"):
			network_snapshots.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	names = dict()
	for relay in relay_perf:
		prefix = relay.stem.replace("_cpu_usage", "")
		name, pid = prefix.split("_")
		names[int(pid)] = name
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			print(f"{name} skipped due to {e}")

	avg_for_pid = average_bandwidth(network_snapshots)
	# fix names
	avg = {names[pid]: v for pid, v in avg_for_pid.items()}


	return sub_avg_latencies, relay_perf_avg, avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/spine-leaf-2-3"))

([0.019263192307692308,
  0.010966411764705884,
  0.012245576923076927,
  0.019375884615384618,
  0.011683921568627452,
  0.012750557692307695,
  0.018563615384615387,
  0.018547156862745093,
  0.012548153846153844,
  0.020063115384615384,
  0.012160942307692307,
  0.019153607843137254,
  0.018999137254901956,
  0.01201894117647059,
  0.018823392156862743,
  0.01923098039215686,
  0.01922734615384615,
  0.019182903846153846,
  0.011916076923076929,
  0.012440576923076922,
  0.011917117647058826,
  0.019140384615384615,
  0.019123705882352945,
  0.012216423076923076,
  0.018602725490196086,
  0.019765627450980386,
  0.01949580769230769,
  0.013286711538461535,
  0.01906069230769231,
  0.018369134615384614,
  0.019183529411764707,
  0.019692411764705875,
  0.019719411764705885,
  0.018868294117647056,
  0.019232941176470585,
  0.013068749999999997,
  0.019759788461538457,
  0.019739000000000003,
  0.018926058823529414,
  0.020309980769230764,
  0.019259431372549023,
  0.01098003846153846

In [53]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")

measurements = []
for topo in ["full-mesh",  "spine-leaf-2-3", "spine-leaf-2-5", "star-4", "star-5", "two-relays", "single-relay"]:
	latencies, relays, avg_bandwidth = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	topo_results = {
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays,
		"average_bandwidth": avg_bandwidth
	}
	measurements.append(topo_results)
	pprint(topo_results)

{'average_bandwidth': {'relay1': (4212.86743428391, 4561.839824235316),
                       'relay2': (4027.4914505591278, 4356.023648794347),
                       'relay3': (4023.2237182227814, 4360.898680274072),
                       'relay4': (4064.3702811661074, 4397.319953962066),
                       'relay5': (4042.8070358051596, 4377.778268793462)},
 'average_latency': 0.015967083636877818,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.34897959183673477,
                        'relay2': 0.2857142857142857,
                        'relay3': 0.3040816326530612,
                        'relay4': 0.31224489795918364,
                        'relay5': 0.3142857142857143}}
{'average_bandwidth': {'leaf1': (5966.104284897295, 6652.787796875022),
                       'leaf2': (5818.536504818777, 6484.696298595258),
                       'leaf3': (5929.25804671175, 6611.673751051477),
                       'spine1': (1218.2453074644304, 1244.9835440878746),
    

In [54]:
for m in measurements:
    print(f"{m['name']}: {round(m['average_latency'] * 1000, 2)}")

full-mesh: 15.97
spine-leaf-2-3: 16.63
spine-leaf-2-5: 15.87
star-4: 13.53
star-5: 14.72
two-relays: 14.84
single-relay: 18.94


In [55]:
# scale CPU utilization to highest
highest_cpu = max(max(t["relay_utilizations"].values()) for t in measurements)

for t in measurements:
	t["average_latency"] = f"{t['average_latency'] * 1000} ms"
	for r, val in t["relay_utilizations"].items():
		t["relay_utilizations"][r] = f"{val / highest_cpu:.0%}"
	for r, val in t["average_bandwidth"].items():
		t["average_bandwidth"][r] = (round(t['average_bandwidth'][r][0],2),round(t['average_bandwidth'][r][1],2))
pprint(measurements)

[{'average_bandwidth': {'relay1': (4212.87, 4561.84),
                        'relay2': (4027.49, 4356.02),
                        'relay3': (4023.22, 4360.9),
                        'relay4': (4064.37, 4397.32),
                        'relay5': (4042.81, 4377.78)},
  'average_latency': '15.967083636877819 ms',
  'name': 'full-mesh',
  'relay_utilizations': {'relay1': '18%',
                         'relay2': '15%',
                         'relay3': '16%',
                         'relay4': '16%',
                         'relay5': '17%'}},
 {'average_bandwidth': {'leaf1': (5966.1, 6652.79),
                        'leaf2': (5818.54, 6484.7),
                        'leaf3': (5929.26, 6611.67),
                        'spine1': (1218.25, 1244.98),
                        'spine2': (985.24, 1001.41)},
  'average_latency': '16.625992369909497 ms',
  'name': 'spine-leaf-2-3',
  'relay_utilizations': {'leaf1': '26%',
                         'leaf2': '23%',
                         'le